# <b>Document AI features demo 2</b>

The AIServiceVisionClient offers the <b>document classification</b>, <b>language classification</b> and <b>table detection</b> features. This notebook aims to provide overall clarity about the feature to the user in terms of requirements, usage and the output of the API.<br>
<ul>
    <li>The raw output is saved as <code>response_document_demo2.json</code> file. </li>
    <li>The detected tables are printed under <b> Display the detected tables</b> section.</li>
    <li>Document classes and detected language codes are printed in descending order of their confidence levels in the last section of the notebook. The language codes follow the ISO 639-2 standard.</li>
    <li>The user can visualize the bounding boxes for the detected tables under <b>View output document with bounding boxes</b> section. </li>
</ul>

### Steps to run the notebook:
<details>
    <summary>Notebook session setup</summary>
    <ol>
        <li><font size="2">Installing the OCI Vision SDK</font></li>
        <li><font size="2">Installing other dependencies</font></li>
        <li><font size="2">Setup sample input documents</font></li>
        <li><font size="2">Setup helper .py files</font></li>
        <li><font size="2">Create output folder</font></li>
    </ol>
</details>

<details>
    <summary>Importing the required modules</summary>
</details>

<details>
    <summary>Setting the input variables</summary>
     <font size="2">The user can change the input variables, if necessary. They have been assigned default values.</font>
</details>

<details>
    <summary>Running the main pipeline</summary>
    <font size="2">Run all cells to get the output in the <code>output</code> directory. </font><br>
</details>

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Vision SDK

In [20]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/vision_service_python_client-0.3.45-py2.py3-none-any.whl"
# !pip install vision_service_python_client-0.3.45-py2.py3-none-any.whl
# !rm vision_service_python_client-0.3.45-py2.py3-none-any.whl

#### Installing other dependencies

In [21]:
# !pip install matplotlib==3.3.4
# !pip install pandas==1.1.5

#### Setup sample input documents

In [22]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/TextDetectionOnePage.pdf"
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/table.pdf"
# !mkdir data
# !mv TextDetectionOnePage.pdf data
# !mv table.pdf data

#### Setup helper .py files

In [23]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/analyze_document_utils.py"
# !mkdir helper
# !mv analyze_document_utils.py helper

#### Create output folder

In [24]:
# !mkdir output

### Imports

In [25]:
import base64
import os
import io
import oci
import json
from IPython.display import IFrame
import requests
import pandas as pd
import numpy as np

from vision_service_python_client.ai_service_vision_client import AIServiceVisionClient
from vision_service_python_client.models.analyze_document_details import AnalyzeDocumentDetails
from vision_service_python_client.models.inline_document_details import InlineDocumentDetails
from vision_service_python_client.models.document_table_detection_feature import DocumentTableDetectionFeature
from vision_service_python_client.models.document_classification_feature import DocumentClassificationFeature
from vision_service_python_client.models.document_language_classification_feature import DocumentLanguageClassificationFeature

from helper.analyze_document_utils import is_url, clean_output, display_classes

### Set input variables
<details>
<summary><font size="3">input_path</font></summary>
<font size="2">The user can provide the document URL or filepath from the notebook session.</font><br>
</details>
<details>
<summary><font size="3">max_results</font></summary>
    <font size="2">Provide the maximum number of results needed for document classification and language classification. This is an upper limit over the output classes, the API may detect lesser classes according to the document.</font><br>
</details>

In [26]:
input_path = "data/table.pdf"
max_results = 5

### Authorize user config

In [27]:
config = oci.config.from_file('~/.oci/config')

### Get input document

In [28]:
if is_url(input_path):
    file_content = requests.get(input_path).content
    encoded_string = base64.b64encode(file_content)
    open('data/' + os.path.basename(input_path), 'wb').write(file_content)
else:
    with open(input_path, "rb") as document_file:
        encoded_string = base64.b64encode(document_file.read())

### View input document

In [29]:
if is_url(input_path):
    display(IFrame('data/' + os.path.basename(input_path), width=600, height=500))
else:
    display(IFrame(input_path, width=600, height=500))

### Create AI service vision client and get response object

In [30]:
ai_service_vision_client = AIServiceVisionClient(config=config)
analyze_document_details = AnalyzeDocumentDetails()
inline_document_details = InlineDocumentDetails()

table_detection_feature = DocumentTableDetectionFeature()
document_classification_feature = DocumentClassificationFeature()
document_classification_feature.max_results = max_results
document_language_classification_feature = DocumentLanguageClassificationFeature()
document_language_classification_feature.max_results = max_results

features = [table_detection_feature, document_classification_feature, document_language_classification_feature]
inline_document_details.data = encoded_string.decode('utf-8')
analyze_document_details.document = inline_document_details
analyze_document_details.features = features

res = ai_service_vision_client.analyze_document(analyze_document_details=analyze_document_details)

### Clean and save the API response as json

In [31]:
res_json = json.loads(repr(res.data))
clean_res = clean_output(res_json)

with open('output/response_document_demo2.json', 'w') as fp:
    json.dump(clean_res, fp)

### Display the detected tables

In [32]:
df = None

for page in clean_res['pages']:
    if 'tables' not in page:
        continue
    for table in page['tables']:
        df = pd.DataFrame(index=np.arange(int(table['row_count'])+1), columns=np.arange(int(table['column_count'])+1)).fillna('')
        if 'body_rows' not in table:
            continue
            
        for row in table['body_rows']:
            if 'cells' not in row:
                continue
                
            for cell in row['cells']:
                row_no = int(cell['row_index'])
                col_no = int(cell['column_index'])
                text = cell['text']
                df.iloc[row_no,col_no] = text
        display(df.style.hide_index())

0,1,2,3
Function,,Algorithms,Applicability
Classification,,Logistic Regression (GLM) Decision Trees Naive Bayes Support Vector Machines (SVM),Classical statistical technique Popular / Rules / tramsparency Embedded app Wide / narrow data/ text
Regrestion,,Linear Regression (GLM) Support Vector Machine (SVM),Classical statistical technique Wide / narrow data/ text
Ancmaly Detection,,One Class SVM,Unknown fraud cases or anomalies
Attribute Importance,,Minimum Description Length (MDL) Principal Components Analysis (PCA),"Attribute reduction, Reduce data noise"
Astociation Rules,,Aeciori,Marker basket analysis / Nest Best Offer
Clustering,,Hierarchicalk-Means HierarchicalO-Claster Expectation-Maximization Clustering (EM),Product grouping/ Text mining Gene and protein analysis
Feature Extraction,,Nornegative Matrie Factoritation (NME) Singular Value Decompostion (SVD),Text analysis / Feature reduction


### Display the document types with their confidence levels

In [33]:
if 'detected_document_types' in clean_res:
    display_classes(clean_res['detected_document_types'], 'Document Type')

,Document Type,Confidence
1,Others,0.999998


### Display the document language codes with their confidence levels

In [34]:
if 'detected_languages' in clean_res:
    display_classes(clean_res['detected_languages'], 'Language Code')

,Language Code,Confidence
1,ENG,1.000000


### View output document with bounding boxes

The user can uncomment and run the cells below to visualize the bounding boxes over the document. This visualization feature is currently supported for <b>PDF format only.</b>

#### Install dependencies

In [35]:
# !pip install fitz==0.0.1.dev2
# !pip install pymupdf==1.18.19

#### Imports

In [36]:
from helper.analyze_document_utils import add_table_bounding_boxes_to_pdf
import fitz

#### Add bounding boxes

In [37]:
doc = fitz.open(input_path)
doc = add_table_bounding_boxes_to_pdf(doc, clean_res)
output_path = 'output/' + 'output_' + os.path.basename(input_path)
doc.save(output_path)

#### Display output document

In [38]:
display(IFrame(output_path, width=600, height=500))